# MUST RUN AT THE START OF EVERYTHING

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import csv
import os

import numpy as np
import tqdm

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.annotations import FeatureAnnotator, LabelAnnotator, load_marginals
from snorkel.learning import SparseLogisticRegression
from snorkel.learning.disc_models.rnn import reRNN
from snorkel.learning.utils import RandomSearch
from snorkel.models import Candidate, FeatureKey, candidate_subclass

In [4]:
edge_type = "dg"

In [5]:
if edge_type == "dg":
    DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])
elif edge_type == "gg":
    GeneGene = candidate_subclass('GeneGene', ['Gene1', 'Gene2'])
elif edge_type == "cg":
    CompoundGene = candidate_subclass('CompoundGene', ['Compound', 'Gene'])
elif edge_type == "cd":
    CompoundDisease = candidate_subclass('CompoundDisease', ['Compound', 'Disease'])
else:
    print("Please pick a valid edge type")

# Load preprocessed data 

To save time, this code will automatically load our labels that were generated in the previous file.

In [6]:
%%time
labeler = LabelAnnotator(lfs=[])

#L_train = labeler.load_matrix(session,split=0)
L_dev = labeler.load_matrix(session,split=1)

3830137it [00:06, 582573.86it/s]


CPU times: user 8.82 s, sys: 1.23 s, total: 10 s
Wall time: 10.4 s


In [ ]:
print "Total Data Shape:"
print L_train.shape
print L_dev.shape
print

In [ ]:
%%time
featurizer = FeatureAnnotator()

F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)

In [ ]:
print "Total Data Shape:"
print F_train.shape
print F_dev.shape
print

# Run Disc Model Classification of Candidates

# Train Logistic Regression Disc Model

In [ ]:
%time train_marginals = load_marginals(session, split=0)

In [ ]:
# Searching over learning rate
param_ranges = {
    'lr' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'l1_penalty' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'l2_penalty' : [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
}
model_hyperparams = {
    'n_epochs' : 50,
    'rebalance' : 0.5,
    'print_freq' : 25
}
searcher = RandomSearch(SparseLogisticRegression, param_ranges, F_train,
                        Y_train=train_marginals, n=5, model_hyperparams=model_hyperparams)

In [ ]:
%%time
np.random.seed(100)
disc_model, run_stats = searcher.fit(F_dev, L_dev, n_threads=4)

In [ ]:
w, b = disc_model.get_weights()

In [ ]:
# Write the weights and features for further processing
annot_select_query = FeatureKey.__table__.select().order_by(FeatureKey.id)
with open("LR_model.csv", "w") as f:
    fieldnames = ["Weight", "Feature"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for weight, feature in tqdm.tqdm(zip(w, session.execute(annot_select_query))):
        writer.writerow({"Weight": weight, "Feature":feature[1]})

## Train Recurrent Neural Net Disc Model

In [6]:
%time train_marginals = load_marginals(session, split=0)
np.savetxt("pmacs/train_marginals", train_marginals)

CPU times: user 11.4 s, sys: 512 ms, total: 11.9 s
Wall time: 19.4 s


In [7]:
%%time
train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 1000,
}

lstm = reRNN(seed=100, n_threads=4)
#lstm.train(train_cands, train_marginals[0:10], X_dev=dev_cands, Y_dev=L_dev[0:10], **train_kwargs)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 98 µs


In [8]:
import csv
chunksize = 100000
start = 0
with open('pmacs/train_candidates_ends.csv', 'wb') as g:
    with open("pmacs/train_candidates_offsets.csv", "wb") as f:
        while True:
            train_cands = session.query(DiseaseGene).filter(DiseaseGene.split == 0).order_by(DiseaseGene.id).limit(chunksize).offset(start).all()
            
            if not train_cands:
                break
                
            output = csv.writer(f)
            for c in tqdm.tqdm(train_cands):
                data, ends = lstm._preprocess_data([c], extend=True)
                output.writerow(data[0])
                g.write("{}\n".format(ends[0]))
            
            start += chunksize

100%|██████████| 83871/83871 [03:30<00:00, 398.77it/s]


In [11]:
import csv
with open("pmacs/train_word_dict.csv", 'w') as f:
    output = csv.DictWriter(f, fieldnames=["Key", "Value"])
    output.writeheader()
    for key in tqdm.tqdm(lstm.word_dict.d):
        output.writerow({'Key':key, 'Value': lstm.word_dict.d[key]})


100%|██████████| 443430/443430 [00:00<00:00, 480707.33it/s]

In [16]:
%%time
dev_cands = session.query(DiseaseGene).filter(DiseaseGene.split == 1).order_by(DiseaseGene.id).all()

CPU times: user 10.6 s, sys: 316 ms, total: 10.9 s
Wall time: 12.8 s


In [19]:
import csv
with open('pmacs/dev_candidates_ends.csv', 'wb') as g:
    with open("pmacs/dev_candidates_offsets.csv", "wb") as f:
        output = csv.writer(f)
        for c in tqdm.tqdm(dev_cands):
            data, ends = lstm._preprocess_data([c])
            output.writerow(data[0])
            g.write("{}\n".format(ends[0]))

100%|██████████| 763802/763802 [33:11<00:00, 383.56it/s]  
